“GLP-1 Use & Outcomes”

The goal is to simulate a realistic healthcare dataset, clean and prepare the data using pandas, and apply data analysis techniques to uncover key trends and insights.
---------------------------------
🧩 Dataset Structure

This analysis uses three related datasets:

1. patients.csv :	Patient demographics and baseline metrics
2. medications.csv :	GLP-1 therapy details and adherence
3. outcomes.csv :	Weight, HbA1c, and clinical outcomes at follow-up

Each dataset includes a common key: patient_id.
----------------------------------

⚙️ Workflow Outline

1. Generate synthetic data using Python.

2. Load and clean data with pandas.

3. Merge datasets to form an analysis-ready table.

4. Conduct exploratory data analysis (EDA) to identify trends.

In [34]:
#generate synthetic data

import pandas as pd
import numpy as np

np.random.seed(42)
n = 1000

# Patients
patients = pd.DataFrame({
    "patient_id": range(1, n+1),
    "age": np.random.randint(25, 80, n),
    "sex": np.random.choice(["M", "F"], n),
    "bmi_baseline": np.random.normal(33, 5, n).round(1),
    "diabetes_type": np.random.choice(["T2DM", "Prediabetes", "None"], n, p=[0.6, 0.25, 0.15]),
    "comorbidities": np.random.choice(["HTN", "HLD", "HTN,HLD", "None"], n),
    "baseline_hba1c": np.random.normal(8.0, 1.5, n).round(1)
})

# Medications
drug_names = ["semaglutide", "liraglutide", "dulaglutide", "none"]
medications = pd.DataFrame({
    "patient_id": patients["patient_id"],
    "drug_name": np.random.choice(drug_names, n, p=[0.4, 0.3, 0.2, 0.1]),
    "start_date": pd.to_datetime("2023-01-01") + pd.to_timedelta(np.random.randint(0, 365, n), unit="D"),
    "adherence_rate": np.random.uniform(0.5, 1.0, n).round(2),
    "duration_days": np.random.randint(90, 540, n),
})
medications["discontinuation_flag"] = medications["adherence_rate"] < 0.7

# Outcomes
outcomes = pd.DataFrame({
    "patient_id": patients["patient_id"],
    "followup_months": np.random.randint(3, 18, n),
    "weight_change_kg": np.random.normal(-5, 4, n).round(1),
    "hba1c_change": np.random.normal(-1.2, 0.7, n).round(2),
    "hospitalization_flag": np.random.choice([0, 1], n, p=[0.85, 0.15])
})
outcomes["outcome_category"] = pd.cut(
    outcomes["hba1c_change"],
    bins=[-10, -1.5, -0.5, 5],
    labels=["Improved", "Stable", "Worsened"]
)

# Save
patients.to_csv("patients.csv", index=False)
medications.to_csv("medications.csv", index=False)
outcomes.to_csv("outcomes.csv", index=False)
print("✅ synthetic GLP1 dataset created!")


✅ synthetic GLP1 dataset created!


In [35]:
#load data into pandas dataframes
patients = pd.read_csv('patients.csv')
meds = pd.read_csv('medications.csv')
outcomes = pd.read_csv('outcomes.csv')

#preview first five rows of dataframes
print(patients.head())
print(meds.head())
print(outcomes.head())


   patient_id  age sex  bmi_baseline diabetes_type comorbidities  \
0           1   63   F          35.8          T2DM           NaN   
1           2   76   M          29.9          T2DM           NaN   
2           3   53   F          30.5          T2DM       HTN,HLD   
3           4   39   M          32.2   Prediabetes           HTN   
4           5   67   M          31.1           NaN           NaN   

   baseline_hba1c  
0            10.6  
1             8.7  
2             7.8  
3             6.9  
4             8.4  
   patient_id    drug_name  start_date  adherence_rate  duration_days  \
0           1  semaglutide  2023-09-30            0.76            343   
1           2  semaglutide  2023-08-20            0.53            233   
2           3  semaglutide  2023-09-13            0.56            113   
3           4  dulaglutide  2023-01-24            0.61            514   
4           5         none  2023-04-19            0.73            323   

   discontinuation_flag  
0     

In [36]:
#combine all columns from patients and meds; used an outer join to retain all records
final = patients.merge(meds, how='outer')
#final.head()

#combine patients and meds with outcomes
final = final.merge(outcomes, how='outer')
final.head()

,patient_id,age,sex,bmi_baseline,diabetes_type,comorbidities,baseline_hba1c,drug_name,start_date,adherence_rate,duration_days,discontinuation_flag,followup_months,weight_change_kg,hba1c_change,hospitalization_flag,outcome_category
0,1,63,F,35.8,T2DM,NaN,10.6,semaglutide,2023-09-30,0.76,343,False,10,-4.7,-0.64,0,Stable
1,2,76,M,29.9,T2DM,NaN,8.7,semaglutide,2023-08-20,0.53,233,True,11,5.2,-2.03,1,Improved
2,3,53,F,30.5,T2DM,"HTN,HLD",7.8,semaglutide,2023-09-13,0.56,113,True,6,-13.1,-0.98,1,Stable
3,4,39,M,32.2,Prediabetes,HTN,6.9,dulaglutide,2023-01-24,0.61,514,True,8,-3.7,-1.07,0,Stable
4,5,67,M,31.1,NaN,NaN,8.4,none,2023-04-19,0.73,323,False,7,-6.8,-1.81,0,Improved


In [ ]:
#number of columns in final dataframe
len(final.columns)

16

In [14]:
#number of rows in final dataframe
len(final)

1000

In [21]:
#check for null values in final dataframe
final[final.isnull().any(axis=1)]


,patient_id,age,sex,bmi_baseline,diabetes_type,comorbidities,baseline_hba1c,drug_name,start_date,adherence_rate,discontinuation_flag,followup_months,weight_change_kg,hba1c_change,hospitalization_flag,outcome_category
0,1,63,F,35.8,T2DM,NaN,10.6,semaglutide,2023-09-30,0.76,False,16,-5.8,-0.52,0,Stable
1,2,76,M,29.9,T2DM,NaN,8.7,semaglutide,2023-08-20,0.53,True,3,0.9,-0.59,1,Stable
4,5,67,M,31.1,NaN,NaN,8.4,none,2023-04-19,0.73,False,7,0.9,-1.45,1,Stable
6,7,45,M,37.0,NaN,"HTN,HLD",8.3,dulaglutide,2023-12-13,0.90,False,12,-4.1,-0.38,0,Worsened
9,10,47,F,31.1,NaN,HLD,7.1,semaglutide,2023-10-15,0.67,True,14,-10.3,-1.18,0,Stable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,990,32,F,34.7,NaN,"HTN,HLD",7.0,semaglutide,2023-01-01,0.51,True,10,2.7,-1.72,1,Improved
992,993,28,M,28.6,Prediabetes,NaN,7.2,dulaglutide,2023-09-28,0.61,True,16,-1.6,-0.54,0,Stable
993,994,49,F,31.5,NaN,HLD,5.6,semaglutide,2023-08-29,0.71,False,16,-3.9,-1.07,0,Stable
996,997,27,F,35.6,T2DM,NaN,10.5,dulaglutide,2023-01-23,0.50,True,16,-8.5,-1.27,0,Stable


In [22]:
final.describe()

,patient_id,age,bmi_baseline,baseline_hba1c,adherence_rate,followup_months,weight_change_kg,hba1c_change,hospitalization_flag
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.00000
mean,500.500000,52.443000,33.020800,8.015600,0.743870,10.103000,-5.06930,-1.182710,0.16300
std,288.819436,15.909993,4.843853,1.495143,0.142898,4.303593,4.23903,0.726254,0.36955
min,1.000000,25.000000,15.600000,3.700000,0.500000,3.000000,-17.90000,-3.890000,0.00000
25%,250.750000,39.000000,29.700000,7.000000,0.620000,6.000000,-8.00000,-1.660000,0.00000
50%,500.500000,53.000000,33.200000,8.000000,0.740000,10.000000,-5.10000,-1.200000,0.00000
75%,750.250000,66.000000,36.200000,9.100000,0.860000,14.000000,-2.20000,-0.690000,0.00000
max,1000.000000,79.000000,47.800000,12.100000,1.000000,17.000000,9.10000,1.160000,1.00000


In [23]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   patient_id            1000 non-null   int64  
 1   age                   1000 non-null   int64  
 2   sex                   1000 non-null   object 
 3   bmi_baseline          1000 non-null   float64
 4   diabetes_type         860 non-null    object 
 5   comorbidities         735 non-null    object 
 6   baseline_hba1c        1000 non-null   float64
 7   drug_name             1000 non-null   object 
 8   start_date            1000 non-null   object 
 9   adherence_rate        1000 non-null   float64
 10  discontinuation_flag  1000 non-null   bool   
 11  followup_months       1000 non-null   int64  
 12  weight_change_kg      1000 non-null   float64
 13  hba1c_change          1000 non-null   float64
 14  hospitalization_flag  1000 non-null   int64  
 15  outcome_category      

In [25]:
# print rows with missing values
final.value_counts()

patient_id  age  sex  bmi_baseline  diabetes_type  comorbidities  baseline_hba1c  drug_name    start_date  adherence_rate  discontinuation_flag  followup_months  weight_change_kg  hba1c_change  hospitalization_flag  outcome_category
3           53   F    30.5          T2DM           HTN,HLD        7.8             semaglutide  2023-09-13  0.56            True                  17               -5.1              -0.95         0                     Stable              1
4           39   M    32.2          Prediabetes    HTN            6.9             dulaglutide  2023-01-24  0.61            True                  10               -2.2              -2.40         1                     Improved            1
6           32   M    27.1          Prediabetes    HTN            6.6             none         2023-07-10  0.72            False                 11               -5.8              -2.06         0                     Improved            1
8           63   F    38.2          Prediabetes    HL

In [ ]:
#how many patients were on each drug

final['drug_name'].value_counts()

drug_name
semaglutide    397
liraglutide    303
dulaglutide    193
none           107
Name: count, dtype: int64

In [ ]:
#average weight change

avg_weight_change = final['weight_change_kg'].mean()
avg_weight_change = round(avg_weight_change,2)
print(f'The average weight change is {avg_weight_change} kg.')

The average weight change is -5.07 kg.


In [ ]:
#total hospitalizations

hospitalizations = final['hospitalization_flag'].sum()
print(f'The total number of hospitalizations is {hospitalizations}.')

The total number of hospitalizations is 158.
